<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [8]'.</span>

<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [6]'.</span>

<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [6]'.</span>

<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [6]'.</span>

In [1]:
from edc import setup_environment_variables
setup_environment_variables()

API credentials have automatically been injected for your active subscriptions.  
The following environment variables are now available:
* `SH_INSTANCE_ID`
* `SH_CLIENT_ID`
* `SH_CLIENT_SECRET`
* `SH_CLIENT_NAME`

------


In [2]:
from edc import check_compatibility
check_compatibility("v0.18.4")

Notebook is compatible


# Impact of Covid19 induced lockdown on NO2 levels 



## Load required dependencies & credentials

In [3]:
# Import required packages

from xcube_sh.config import CubeConfig
from xcube_sh.cube import open_cube
from xcube_sh.sentinelhub import SentinelHub
import xarray as xr

from sentinelhub import BBox, WmsRequest, DataSource, SHConfig
from functools import partial

import os
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt


In [4]:
# Set Sentinel Hub credentials

sh_credentials = dict(client_id='<your Oauth client id>',
                      client_secret='<client secret >') # This is only provided when the Oauth credentials are created

# Sentinel-3 OLCI, Sentinel-3 SLSTR and Sentinel-5 layers are processed on different infrastructure, 
# which requires to used different end-point

sh_credentials.update(api_url='https://creodias.sentinel-hub.com/api/v1') 

## Core function to calculate NO2 values

NO2 values are calculated over the given period (Jan -May) for 2020 & 2019

In [5]:
def caculateNO2(geometry, year):
    """
    parameters:
    geometry: BBox object to be passed. This contains the bounding box for the area of interest (AOI)
    year: string object with the year for which we are calculating the data YYYY
    
    return:
    a dataframe with two columns: Mean NO2 and Timestamp
    NO2 mean values for the time span between Jan 01 to May 29th of the given ''year''
    
    """
    cube_config = CubeConfig(dataset_name='S5PL2',
                         band_names=['NO2'],
                         tile_size=[512, 512],
                         geometry=geometry,
                         spatial_res=abs(bbox[2]-bbox[0])/512,
                         time_range=[year+'-01-01', year+'-05-29'],
                         time_period='3D') 
    cube = open_cube(cube_config, **sh_credentials)

    no2_values = list() 
    timestamp = list()

    for i in range(cube.time.shape[0]):
        no2_values.append(np.nanmean(cube.NO2.isel(time=i).values[0]))
        timestamp.append(cube.NO2.isel(time=i).time.values)
        
    assert len(no2_values) == len(timestamp)
    
    return pd.DataFrame({'DateTime': timestamp, 'Mean NO2': no2_values})

## Read area of interest as a csv file
The file has the following columns:
Country: Name of country/territory of interest
x1, y1 : Upper left corner co-ordinates (lat/lon)
x2, y2 : Lower right corner co-ordinates


In [6]:
aoi = pd.read_csv(os.path.expanduser('~/.shared/artifacts/NO2_Analysis_Covid19_Lockdowns-AOI.csv'))
print(aoi.shape)
aoi.head(8)

(16, 5)


,Country,x1,y1,x2,y2
0,Austria,48.622,11.821,46.561,16.963
1,Belgium,51.544,2.661,49.831,6.550
2,Bulgaria,44.099,22.421,41.503,28.332
3,Crotia,46.619,13.250,42.647,18.600
4,Czech,51.096,12.338,48.573,18.622
5,France,50.720,-4.110,42.770,8.260
6,Germany,54.770,4.700,47.630,15.730
7,Greece,41.500,18.460,35.010,27.910


In [7]:
shape = aoi.shape
nullCount = sum(aoi.isna().sum())
print(f"Shape of 2020 DF: {shape}, Count of Null values: {nullCount}".format(shape , nullCount ))

Shape of 2020 DF: (16, 5), Count of Null values: 0


## Main loop 

We iterate over each row in the `aoi.csv` file 

Sentinel-5P data for each defined AOI cube is extracted 

NO2 values for each tile is then averaged & stored in a list

In [8]:
import time 

start = time.time()
aoi_no2 = list()
for idx, row in aoi[7:].iterrows():
    aoi_dict = dict()
    aoi_dict['Country_BBox'] = row['Country']
    print("Processing: ", aoi_dict['Country_BBox'])
    x1 = row['x1']  # degree 
    y1 = row['y1']  # degree
    x2 = row['x2']  # degree
    y2 = row['y2']  # degree

    bbox = x1, y1, x2, y2
    aoi_dict['NO2_2020'] = caculateNO2(bbox, '2020')
    shape = aoi_dict['NO2_2020'].shape
    nullCount = sum(aoi_dict['NO2_2020'].isna().sum())
    print(f"Shape of 2020 DF: {shape}, Count of Null values: {nullCount}".format(shape , nullCount ))    
    
    aoi_dict['NO2_2019'] = caculateNO2(bbox, '2019')
    shape = aoi_dict['NO2_2019'].shape
    nullCount = sum(aoi_dict['NO2_2019'].isna().sum())
    print(f"Shape of 2019 DF: {shape}, Count of Null values: {nullCount}".format(shape , nullCount ))
    
    aoi_no2.append(aoi_dict)

end = time.time()
print(len(aoi_no2))
print(end-start)

Processing:  Greece


CustomOAuth2Error: ({'status': 400, 'reason': 'Bad Request', 'message': 'Illegal client_id', 'code': 'OAUTH_ERROR'}) 

The list is then pickled so that the information can then be reused in another run

In [ ]:
import pickle 
print(len(aoi_no2))
with open('aoi_list_7.pkl', 'wb') as f:
    pickle.dump(aoi_no2, f)

## Plotting function

This plots the NO2 values over time

In [ ]:
import matplotlib as plt

def plot_no2(idx):
    """
    parameters:
    idx: Input id for accessing NO2 data 
    
    return:
    None
    line plots for NO2 (2020 vs 2019) are generated
    """

    plt.plot(aoi_no2[idx]['NO2_2020']['DateTime'], aoi_no2[idx]['NO2_2020']['Mean NO2'],  label = 'NO2 levels (2020)')
    plt.plot(aoi_no2[idx]['NO2_2020']['DateTime'], aoi_no2[idx]['NO2_2019']['Mean NO2'], label = 'NO2 levels (2019)')
    plt.ylabel('NO2 Levels')
    plt.xlabel('Month')
    plt.title('NO2 levels for '+ aoi_no2[idx]['Country_BBox'])
    plt.legend()
    plt.savefig(aoi_no2[idx]['Country_BBox']+'.png')
    plt.show()

In [ ]:
for idx in range(len(aoi_no2)):
    plot_no2(idx)
